In [1]:
lineList = [line.rstrip('\n') for line in open('all_linkedin_skill_data.txt','r')]

In [2]:
import pandas as pd

In [3]:
train=pd.DataFrame(lineList)

In [4]:
train.head()

,0
0,"[[], [""Contract Negotiation"", ""Supply Chain Ma..."


In [5]:
lists = train[0][0]

In [6]:
res = lists.strip('][').split('], [') 

In [7]:
res[0]=res[0].strip(', [')

In [8]:
train2=pd.DataFrame(res)

In [9]:
train2.columns=['skills']

In [10]:
train2.head(10)

,skills
0,"""Contract Negotiation"", ""Supply Chain Manageme..."
1,"""projetista de maquinas"", ""tecnico eletrico"", ..."
2,
3,"""Recruiting"", ""Human Resources"", ""Talent Acqui..."
4,"""Well Intervention"", ""Project Management"", ""Pr..."
5,
6,"""AML"", ""Risk Management"", ""Banking"", ""Risk Ass..."
7,
8,
9,


In [11]:
import numpy as np

In [12]:
train2 = train2.replace(r'^\s*$', np.nan, regex=True)

In [13]:
train2.tail(10)

,skills
2985323,"""Research"", ""Entrepreneurship"", ""Project Manag..."
2985324,"""Solidworks"", ""Pro Engineer"", ""solidworks simu..."
2985325,"""Entrepreneurship"", ""Leadership"", ""Teaching"", ..."
2985326,"""Eclipse"", ""Scrum"", ""Java"", ""Software Developm..."
2985327,"""Business Intelligence"", ""Solution Architecture"""
2985328,"""Art Direction"", ""Visual Development"", ""Creati..."
2985329,NaN
2985330,"""Urban Planning"", ""Microsoft Office"""
2985331,"""Turnkey Projects"", ""Renewable Energy"", ""Power..."
2985332,"""Business Intelligence"", ""Solution Selling"", ""..."


In [14]:
train2.isna().sum()

skills    1495065
dtype: int64

In [15]:
train=train2.dropna()

In [16]:
train.head(10)

,skills
0,"""Contract Negotiation"", ""Supply Chain Manageme..."
1,"""projetista de maquinas"", ""tecnico eletrico"", ..."
3,"""Recruiting"", ""Human Resources"", ""Talent Acqui..."
4,"""Well Intervention"", ""Project Management"", ""Pr..."
6,"""AML"", ""Risk Management"", ""Banking"", ""Risk Ass..."
11,"""PHP"", ""MySQL"", ""HTML5"", ""CSS3"", ""jQuery"", ""Ja..."
16,"""Engineering"", ""Supply Chain Management"", ""Qua..."
17,"""Ethernet"", ""Embedded Systems"", ""Telecommunica..."
18,"""Hospitals"", ""Networking"", ""Insurance"", ""Healt..."
21,"""Cement"", ""Stimulation"", ""Offshore"", ""Completi..."


In [21]:
documents = [_text.replace('"','').split(',') for _text in train.skills] 

In [22]:
documents

[['Contract Negotiation',
  ' Supply Chain Management',
  ' Cost Reduction',
  ' Strategic Sourcing',
  ' Materials',
  ' Subcontracting',
  ' Sourcing & Procurement',
  ' Intercultural Communication',
  ' Global Sourcing',
  ' Materials Management',
  ' Logistics',
  ' Equipment Procurement',
  ' Purchasing Negotiations',
  ' Supplier Management',
  ' Supplier Evaluation',
  ' Supplier Negotiations'],
 ['projetista de maquinas', ' tecnico eletrico', ' tecnica mecanica'],
 ['Recruiting',
  ' Human Resources',
  ' Talent Acquisition',
  ' Employee Engagement',
  ' Sourcing',
  ' Executive Search',
  ' Technical Recruiting',
  ' Vendor Management',
  ' Benefits Negotiation'],
 ['Well Intervention',
  ' Project Management',
  ' Production Technology',
  ' Product Optimization',
  ' Sand Control',
  ' Completions',
  ' Modeling',
  ' Field Development',
  ' Petroleum Engineering',
  ' Production Engineering'],
 ['AML',
  ' Risk Management',
  ' Banking',
  ' Risk Assessment',
  ' Internal 

In [18]:
p=documents[:20]

In [23]:
from gensim.models import Word2Vec, FastText

In [24]:
word2vec=Word2Vec(documents,min_count=1, sg=1)
# Couldn't use FastText because their was not enough space to store the prepared weights, the array was of (2000000,) size
# Tried a few other embedding techniques but obtained poor results or taking large enough time for colab to crash
# model_ted = FastText(p, size=100, window=5, min_count=5, workers=4,sg=1)

In [25]:
len(word2vec.wv.vocab)

721028

In [26]:
word2vec.wv.most_similar("SQL")

[(' SQL', 0.8492921590805054),
 ('SQL Server', 0.8302315473556519),
 ('Oracle SQL', 0.8223612308502197),
 (' LogiXML', 0.8204731941223145),
 ('PL/SQL', 0.8152726292610168),
 ('Microsoft SQL Server', 0.8088290691375732),
 ('Crystal Reports', 0.8087397813796997),
 (' Dbase', 0.8059900403022766),
 (' 4GL', 0.8043805360794067),
 (' LANSA', 0.8022280931472778)]

In [27]:
import warnings
warnings.filterwarnings(action='ignore')


import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
tf.logging.set_verbosity(tf.logging.ERROR)

In [28]:
import os

In [30]:
dim = len(word2vec.wv.vocab)
embedding = np.empty((len(word2vec.wv.vocab), word2vec.layer1_size), dtype=np.float32)
for i, word in enumerate(word2vec.wv.vocab):
    embedding[i] = word2vec.wv[word]

# setup a TensorFlow session                                                                                                       
tf.reset_default_graph()
sess = tf.InteractiveSession()
X = tf.Variable([0.0], name='browse_history_embedding')

place = tf.placeholder(tf.float32, shape=embedding.shape)
set_x = tf.assign(X, place, validate_shape=False)
sess.run(tf.global_variables_initializer())
sess.run(set_x, feed_dict={place: embedding})

# write labels                                                                                                                     
with open("projections/metadata.tsv", 'w+') as f:
    for word in word2vec.wv.vocab:
        f.write(word + '\n')

# create a TensorFlow summary writer                                                                                               
summary_writer = tf.summary.FileWriter('projections', sess.graph)
config = projector.ProjectorConfig()
embedding_conf = config.embeddings.add()
embedding_conf.tensor_name = 'browse_history_embedding:0'
embedding_conf.metadata_path = os.path.join('projections', 'metadata.tsv')
projector.visualize_embeddings(summary_writer, config)

# save the model
saver = tf.train.Saver()
saver.save(sess, os.path.join('projections', "model.ckpt"))

'projections/model.ckpt'

In [ ]:
# Now, open the terminal and type the following command to open the tensorboard,

# tensorboard --logdir=projections --port=8000